In [44]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [45]:
text_file = "fcc-gpt-course/wizard_of_oz.txt"

with open(text_file, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(set(text))
print(chars)

vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [46]:
# tokenizers

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

# encoder
encode = lambda s: [string_to_int[c] for c in s]
# decoder
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [47]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)

    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[71,  1, 65, 68, 67, 60,  1, 54],
        [ 1, 72, 68,  0, 73, 68, 68,  9],
        [73,  1, 54, 67, 57,  1, 56, 68],
        [58,  1, 76, 62, 65, 65,  1, 72]])
targets:
tensor([[ 1, 65, 68, 67, 60,  1, 54, 71],
        [72, 68,  0, 73, 68, 68,  9,  1],
        [ 1, 54, 67, 57,  1, 56, 68, 75],
        [ 1, 76, 62, 65, 65,  1, 72, 73]])


In [48]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [49]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


bq9_Jf(y:fQhds"R&Uq,8TQZBm?YRL0iC_nT'2rE:iB3Y;s:!(57hoi4'zhJSaV9!FMBC:DC4lQe9Z!MnWf-p-21p&3l!Fj:yUvIKcdH&7"rmcgy'jiZtFWTG?H9-[9;U_sO4Kj"muN31TNK"lwN*T3.B2a0f;Xv5S
0z.C1F(;9'ogiEBT7
('FDmvI0zRlx1pF(irFBRb
[p&xSJaYRJ&W *tadsRlW!XYYS v_]aDE]*UDC11L
n7Ba(y_RKHe3MDCCMo;-VWQDCzXD,XWLFw"sA-ntFwO'ebAWQsA-7ZMLYEQBMF)Q*WWr,fYSA5hKr-[WTM6roRY"5H"PV9﻿2qg4A48uG7lx(0Alu4;-WIH
0VHRa74aJ"ZM9h"lv9QitME,fN yf088[Nmq [(BRtst)0pj.zF(Y
BRh4CVwI';G(;TRnlLn4F8;h]!jUnRaDc"WWTHeAJkCSCPf,5(qOHJ)Q8.7iFbl2'Xoi)D,6.1p?mMre0


In [50]:
@torch.no_grad()
def estimate_loss():
    out = {}
    
    model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        
        out[split] = losses.mean()
    
    model.train()
    
    return out

In [51]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for it in range(max_iters):
    if it % eval_iters == 0:
        losses = estimate_loss()
        print(f'step : {it}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    optimizer.step()

print(loss.item())

step : 0, train loss: 4.960, val loss: 4.943
step : 250, train loss: 4.880, val loss: 4.897


step : 500, train loss: 4.819, val loss: 4.817
step : 750, train loss: 4.745, val loss: 4.764
4.509397506713867


1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.

2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function

3. **Momentum**: Momentum is an extension of SGD that adds a \"momentum\" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.

4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.

5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.

6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [52]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


6G9tLCWW&W&s-[I_
_s_ PTSh"td45 "p;q uktd:
**_6AR1h31pqmEKyiI0SYg1aM9zRh)Qb tm?oPxPZMz;siPiYxplbr'.CVOa]V5iSBw﻿P*?2p0j1a54Y6&l(6!P
0O!E1piF:A;emm;[5XyAE*yTiw-!e1y:Cch3D"WN0!(2fHlu8F
﻿E3j(:Ud*_[d)o.)0kG? bbqNb8YmocuTBBx5;XW.Fhv3'0!d_-?92WI".OZ-v;yF
rMhZg[ZMUA;-IyAPT_[[q.P]n71B8T3L,JT; g526'. 
g1pKm8uTDj*2RxvPBwj1p&W3f
OCjG]o2j::vsBmytas"?.x'[vwc FU9lImqy7ola,fs:;4lOQJXv[99X8_?,L:OwOKnQn1N&Xr﻿2V:f[ByUzohkTr48DmuGIy'oc[2igVR3﻿YXje9-vI"Pp*2MB8q,camD'wgP:kuZMz
(-.3pca(X;-v_l?x00!)],Js1)d_pDry56(A;I0sR
